In [1]:
#IMPORTING FULL DATASETS  
import numpy as np
import h5py

with h5py.File('Final Dataset.h5', 'r') as hdf:
    X_train_full_temp = hdf.get('X_train_full')
    y_train_full_temp = hdf.get('y_train_full')
    X_test_full_temp = hdf.get('X_test_full')
    y_test_full_temp = hdf.get('y_test_full')
    
    X_train_full = np.array(X_train_full_temp)
    y_train_full = np.array(y_train_full_temp)
    X_test_full = np.array(X_test_full_temp)
    y_test_full = np.array(y_test_full_temp)

In [2]:
print(X_train_full.shape)
print(y_train_full.shape)
print(X_test_full.shape)
print(y_test_full.shape)

(90000, 28, 28)
(90000,)
(16000, 28, 28)
(16000,)


In [3]:
#NORMALISING
X_train_full = X_train_full.astype(np.float32)/ 255.0
X_test_full = X_test_full.astype(np.float32)/ 255.0

In [4]:
#RESHAPING DATA TO HAVE A SINGLE COLOUR CHANNEL
X_train_full = X_train_full.reshape((X_train_full.shape[0], 28, 28, 1))
X_test_full = X_test_full.reshape((X_test_full.shape[0], 28, 28, 1))
print(X_train_full.shape)
print(X_test_full.shape)

(90000, 28, 28, 1)
(16000, 28, 28, 1)


In [5]:
#PADDING IMAGES WITH 0'S TO MAKE 32X32 PIXEL IMAGES
X_train_full = np.pad(X_train_full, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test_full = np.pad(X_test_full, ((0,0),(2,2),(2,2),(0,0)), 'constant')
print(X_train_full.shape)
print(X_test_full.shape)

(90000, 32, 32, 1)
(16000, 32, 32, 1)


In [6]:
#CREATING A VALIDATION SET
X_valid, X_train = X_train_full[:20000], X_train_full[20000:]
y_valid, y_train = y_train_full[:20000], y_train_full[20000:]
print(X_train.shape)
print(X_valid.shape)

(70000, 32, 32, 1)
(20000, 32, 32, 1)


In [7]:
#CHANGING THE LABEL DATA TO CATEGORICAL
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)
y_test = to_categorical(y_test_full)

In [8]:
#BUILIDING A LeNet-5 NEURAL NETWORK 
import tensorflow as tf
from tensorflow import keras
from functools import partial

model = keras.models.Sequential([
    keras.layers.Conv2D(filters=6, kernel_size=(3,3), activation='relu', input_shape=[32, 32, 1]),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu'),
    keras.layers.AveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(units=120, activation='relu'),
    keras.layers.Dense(units=84, activation='relu'),
    keras.layers.Dense(units=16, activation='softmax'),
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 6)         60        
_________________________________________________________________
average_pooling2d (AveragePo (None, 15, 15, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 16)        880       
_________________________________________________________________
average_pooling2d_1 (Average (None, 6, 6, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               69240     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [10]:
#COMPILING MODEL
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adam(), 
              metrics=['accuracy'])

In [11]:
# TRAINING THE NERUAL NETWORK CLASSIFIER
history = model.fit(X_train, y_train, epochs=100,
                    batch_size=100,
                    validation_data=(X_valid, y_valid), 
                    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
                   )

Epoch 1/100
700/700 [==============================] - 16s 22ms/step - loss: 0.8213 - accuracy: 0.7530 - val_loss: 0.1280 - val_accuracy: 0.9636
Epoch 2/100
700/700 [==============================] - 15s 21ms/step - loss: 0.1129 - accuracy: 0.9664 - val_loss: 0.0839 - val_accuracy: 0.9736
Epoch 3/100
700/700 [==============================] - 15s 21ms/step - loss: 0.0726 - accuracy: 0.9785 - val_loss: 0.0519 - val_accuracy: 0.9851
Epoch 4/100
700/700 [==============================] - 14s 21ms/step - loss: 0.0523 - accuracy: 0.9838 - val_loss: 0.0518 - val_accuracy: 0.9840
Epoch 5/100
700/700 [==============================] - 15s 21ms/step - loss: 0.0422 - accuracy: 0.9876 - val_loss: 0.0455 - val_accuracy: 0.9865
Epoch 6/100
700/700 [==============================] - 15s 21ms/step - loss: 0.0374 - accuracy: 0.9883 - val_loss: 0.0355 - val_accuracy: 0.9885
Epoch 7/100
700/700 [==============================] - 15s 21ms/step - loss: 0.0294 - accuracy: 0.9914 - val_loss: 0.0303 - val_ac

In [12]:
#EVALUATING ON TEST SET
score = model.evaluate(X_test_full, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

500/500 [==============================] - 2s 4ms/step - loss: 0.0224 - accuracy: 0.9941
Test loss: 0.0223829448223114
Test accuracy: 0.9941250085830688


In [13]:
#SAVING MODEL
model.save("Trained Model.h5")
print("Saved model to disk")

Saved model to disk
